### Install & Import Packages

In [ ]:
%%capture
!pip install "flair" -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import flair
from flair.data import Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import (
    WordEmbeddings, FlairEmbeddings, StackedEmbeddings
)
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
flair.__version__

'0.12.2'

### Import Data

In [ ]:
DATA_PATH = "/content/drive/Shareddrives/CIS522-Project/data"
MODEL_PATH = "/content/drive/Shareddrives/CIS522-Project/models"

In [ ]:
# Import data into flair using ColumnCorpus
corpus = ColumnCorpus(DATA_PATH, {0: "text", 1: "ner"}, train_file="flair_ner_train.txt", test_file="flair_ner_test.txt")
# Needed for model initialization
tag_dictionary = corpus.make_label_dictionary(label_type="ner", add_unk=False)
print(tag_dictionary.get_items())

2023-04-18 23:04:18,219 Reading data from /content/drive/Shareddrives/CIS522-Project/data
2023-04-18 23:04:18,227 Train: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_train.txt
2023-04-18 23:04:18,230 Dev: None
2023-04-18 23:04:18,231 Test: /content/drive/Shareddrives/CIS522-Project/data/flair_ner_test.txt
2023-04-18 23:05:10,425 Computing label dictionary. Progress:


35399it [00:01, 27982.82it/s]

2023-04-18 23:05:11,724 Dictionary created for label 'ner' with 9 values: Drug (seen 84430 times), Strength (seen 59496 times), Form (seen 56512 times), Frequency (seen 49151 times), Route (seen 40294 times), Dosage (seen 32719 times), Reason (seen 13209 times), Duration (seen 3233 times), ADE (seen 1988 times)
['Drug', 'Strength', 'Form', 'Frequency', 'Route', 'Dosage', 'Reason', 'Duration', 'ADE']


### Initialize Weight Dictionary
This is the weight dictionary used by the loss function. The weight for a given entity is set to the ratio between the frequency of the most represented entity and the frequency of the given entity.

In [ ]:
weight_dict = {
    'Drug': 84430/84430,
    'Strength': 84430/59496,
    'Form': 84430/56512,
    'Frequency': 84430/49151,
    'Route': 84430/40294,
    'Dosage': 84430/32719,
    'Reason': 84430/13209,
    'Duration': 84430/3233,
    'ADE': 84430/1988,
}

In [ ]:
# Example ColumnCorpus train sample
corpus.train[0]

Sentence[38]: "Since no new infection was found this was presumed [ * * 12-26 * * ] steroids and the leukocytosis improved with prednisone taper . WBC 12 on day of discharge . Hyperglycemia : Patient is not known" → ["steroids"/Drug, "leukocytosis"/ADE, "prednisone"/Drug, "Hyperglycemia"/ADE]

In [ ]:
# Example ColumnCorpus test sample
corpus.test[0]

Sentence[14]: "MEDICATIONS : Lipitor , Tylenol with Codeine , Dilantin , previously on Decadron q.i.d" → ["Lipitor"/Drug, "Tylenol with Codeine"/Drug, "Dilantin"/Drug, "Decadron"/Drug, "q.i.d"/Frequency]

### Initialize/Load Embeddings & Model

In [ ]:
# RUN ONLY DURING EMBEDDINGS/MODEL INITIALIZATION 
# embedding_types = [
#     # word embeddings trained on PubMed and PMC
#     WordEmbeddings("pubmed"),
#     # flair embeddings trained on PubMed and PMC
#     FlairEmbeddings("pubmed-forward"),
#     FlairEmbeddings("pubmed-backward"),
# ]
# embeddings = StackedEmbeddings(embeddings=embedding_types)
# tagger = SequenceTagger(
#     hidden_size=256,
#     embeddings=embeddings,
#     tag_dictionary=tag_dictionary,
#     tag_type="ner",
#     rnn_type='LSTM',
#     tag_format='BIOES',
#     use_crf=True,
#     locked_dropout=0.5,
#     loss_weights=weight_dict
# )

# RUN WHEN A COPY OF THE MODEL HAS BEEN SAVED TO DRIVE
tagger = SequenceTagger.load(f"{MODEL_PATH}/taggers/lstm-crf/final-model.pt")

2023-04-18 22:58:46,483 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


### Train Model

In [ ]:
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)

# Train on corpus
trainer.train(
    base_path=f"{MODEL_PATH}/taggers/lstm-crf",
    train_with_dev=False,
    max_epochs=10,
    learning_rate=0.1,
    mini_batch_size=64,
    embeddings_storage_mode='none'
)

### Evaluate Model

In [ ]:
tagger = SequenceTagger.load(f"{MODEL_PATH}/taggers/lstm-crf/final-model.pt")
result = tagger.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=64)
print(result.detailed_results)

2023-04-18 23:06:04,517 SequenceTagger predicts: Dictionary with 39 tags: O, S-Drug, B-Drug, E-Drug, I-Drug, S-Strength, B-Strength, E-Strength, I-Strength, S-Form, B-Form, E-Form, I-Form, S-Frequency, B-Frequency, E-Frequency, I-Frequency, S-Route, B-Route, E-Route, I-Route, S-Dosage, B-Dosage, E-Dosage, I-Dosage, S-Reason, B-Reason, E-Reason, I-Reason, S-Duration, B-Duration, E-Duration, I-Duration, S-ADE, B-ADE, E-ADE, I-ADE, <START>, <STOP>


100%|██████████| 397/397 [05:26<00:00,  1.22it/s]


2023-04-18 23:11:32,655 Evaluating as a multi-label problem: False

Results:
- F-score (micro) 0.9095
- F-score (macro) 0.8385
- Accuracy 0.8397

By class:
              precision    recall  f1-score   support

        Drug     0.9015    0.9395    0.9201     61167
    Strength     0.9448    0.9588    0.9517     42957
        Form     0.9209    0.9292    0.9250     41417
   Frequency     0.8319    0.8416    0.8367     36495
       Route     0.9436    0.9620    0.9527     30583
      Dosage     0.9279    0.9404    0.9341     23506
      Reason     0.7458    0.7745    0.7598      9533
    Duration     0.7724    0.7926    0.7824      1982
         ADE     0.4158    0.5781    0.4837      1299

   micro avg     0.8991    0.9202    0.9095    248939
   macro avg     0.8227    0.8574    0.8385    248939
weighted avg     0.9001    0.9202    0.9100    248939



In [ ]:
# Create example sentence
sentence = Sentence("Patients on 40 mg of Topelfate and Topoxy twice a day generally suffer from headache")

# Token level predictions
tagger.predict(sentence, force_token_predictions=True)
print(sentence.to_tagged_string())

# Predict tags and print
tagger.predict(sentence)
print(sentence.to_tagged_string())